In [8]:
from transformers import BertTokenizer, AutoConfig, AutoModelForSequenceClassification 

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

loading file https://huggingface.co/bert-base-cased/resolve/main/vocab.txt from cache at C:\Users\denni/.cache\huggingface\transformers\6508e60ab3c1200bffa26c95f4b58ac6b6d95fba4db1f195f632fa3cd7bc64cc.437aa611e89f6fc6675a049d2b5545390adbc617e7d655286421c191d2be2791
loading file https://huggingface.co/bert-base-cased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-cased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-cased/resolve/main/tokenizer_config.json from cache at C:\Users\denni/.cache\huggingface\transformers\ec84e86ee39bfe112543192cf981deebf7e6cbe8c91b8f7f8f63c9be44366158.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at C:\Users\denni/.cache\huggingface\transformers\a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3e

In [2]:
from glob import glob
from datasets import Dataset

import re
import pandas as pd

def data_loading(url):
    with open(url, 'r', encoding='utf-8') as f:
        # data = json.loads(f.read())
        df = pd.read_json(f)
        data = df.copy()
        # data = df.loc[:, ['sentiment', 'body']]
        data = data.loc[df['sentiment'].notnull()]
        data['sentiment'] = pd.Categorical(data['sentiment'])
        data['label'] = data['sentiment'].cat.codes
        data = data.rename(columns={'sentiment': 'labels', 'body': 'sentense'})

        return data

data_url = '../crawler/stock/data/**.json'
url = glob(data_url)[-1]
data = data_loading(url)

dataset = Dataset.from_pandas(data.loc[:, ['label', 'sentense']])
dataset = dataset.remove_columns('__index_level_0__')

In [3]:
def encode(example):
    result = tokenizer(example['sentense'], padding='max_length', truncation=True)
    
    return result

encoded_dataset = dataset.map(encode, batched=True)
encoded_dataset = encoded_dataset.train_test_split(test_size=0.2)
# print(encoded_dataset[0]['sentense'])
# print(encoded_dataset[0]['input_ids'])
# type(encoded_dataset[0]['labels'])

100%|██████████| 1/1 [00:00<00:00,  1.08ba/s]


In [4]:
from transformers import TrainingArguments, Trainer
from datasets import load_metric

import numpy as np

training_args = TrainingArguments(
    output_dir="bert-classifier", 
    evaluation_strategy="epoch",
    save_steps=50,
    num_train_epochs=1
)

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['test'],
    compute_metrics=compute_metrics,
)

trainer.train()


The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentense. If sentense are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
C:\Users\denni\anaconda3\envs\stock-nlp\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 356
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 45
100%|██████████| 45/45 [21:55<00:00, 25.32s/it]The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.

{'eval_loss': 0.6272643804550171, 'eval_accuracy': 0.7111111111111111, 'eval_runtime': 158.4006, 'eval_samples_per_second': 0.568, 'eval_steps_per_second': 0.076, 'epoch': 1.0}
{'train_runtime': 1474.0269, 'train_samples_per_second': 0.242, 'train_steps_per_second': 0.031, 'train_loss': 0.5318872239854601, 'epoch': 1.0}


TrainOutput(global_step=45, training_loss=0.5318872239854601, metrics={'train_runtime': 1474.0269, 'train_samples_per_second': 0.242, 'train_steps_per_second': 0.031, 'train_loss': 0.5318872239854601, 'epoch': 1.0})

In [7]:
model.save_pretrained('./bert-classifier/test-epoch-1')

Configuration saved in ./bert-classifier/test-epoch-1\config.json
Model weights saved in ./bert-classifier/test-epoch-1\pytorch_model.bin
